In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

### Preprocessing of Pre-trained Experimental Electrical Resistivity Data

In [2]:
# load experimental dataset used for the model training
train = pd.read_csv('./csv/Train.csv')
val = pd.read_csv('./csv/Val.csv')
total = pd.concat([train, val], axis=0).reset_index(drop=True)

# Intrinsic and extrinsic features
# t: thickness of the film [nm], depo: deposition rate [nm/s], 
# r: Average atomic radius [], del_r: Atomic radius mismatch, del_EN: Electronegativity difference, 
# S: Mixing entropy [J/mol-K], H: Heat of mixing [kJ/mol], VEC: Average valence electron concentration, Th: Homologous temperature
# Formulas for input compositional features can be found in methods section of the manuscript
norm_list = ['t', 'depo', 'r', 'del_r', 'del_EN', 'S', 'H', 'VEC', 'Th']
normalize = total[norm_list]

# Normalization
scaler = MinMaxScaler()
normalize[:] = scaler.fit_transform(normalize[:])

### Predicting Electrical Resistivity of Alloys

In [8]:
# Load compositional features and thickness of unseen alloy compositions
file_name = 'Test_features' # Copy and input compositional feature you want to evaluate
df = pd.read_csv(f'./csv_test/{file_name}.csv')

# Calculate the features which is determined by the sputtering conditions
df['depo'] = df['t'] / 3000 # divide thickness by deposition time [sec]
df['Th'] = 773 / df['Tm'] # Tsub is the substrate temperature [K], you set during film deposition

# Normalization using scaler fitted for features used for the model training
df_norm = df.loc[:, norm_list]
df_norm[:] = scaler.transform(df_norm[:])

# Load pre-trained electrical resistivity prediction model
model = tf.keras.models.load_model(f'./model/ANN.h5', compile=False)

# Predict resistivity of unseen alloy compositions and save in result folder
pred = model(tf.convert_to_tensor(df_norm, dtype=tf.float32))
pred_df = pd.DataFrame(pred.numpy(), columns=['resistivity_pred'])
pd.concat([df, pred_df], axis=1).to_csv(f'./result/{file_name}_pred.csv', index=False)